In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [7]:
import numpy as np
import torch
import torch.nn as nn
import gym
import gym.spaces

In [11]:
environment = gym.make("FrozenLake-v1", is_slippery=False)

In [25]:
environment.action_space, environment.observation_space.n, 
#environment.env.P

(Discrete(4), 16)

In [26]:
class OneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(OneHotWrapper,self).__init__(env)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype = np.float32)

    def observation(self, observation):
        r = np.copy(self.observation_space.low)
        r[observation] = 1.0
        return r

environment = OneHotWrapper(environment)

# class OneHotWrapper(gym.ObservationWrapper):
# def __init__(self, env):
#    super(OneHotWrapper, self).__init__(env)
#    self.observation_space = gym.spaces.Box(0.0, 1.0, 
#               (env.observation_space.n, ), dtype=np.float32)
# def observation(self, observation):
#     r = np.copy(self.observation_space.low)
#     r[observation] = 1.0
#     return r

# env = OneHotWrapper(env)

In [131]:
obs_size = environment.observation_space.shape[0]
n_actions = environment.action_space.n

print(obs_size, n_actions)

HIDDEN_SIZE = 32

net = nn.Sequential(
    nn.Linear(obs_size, HIDDEN_SIZE),
    nn.ReLU(),
    nn.Linear(HIDDEN_SIZE, n_actions)
)

# obs_size = env.observation_space.shape[0]
# n_actions = env.action_space.n
# HIDDEN_SIZE = 32
# net= nn.Sequential(
#      nn.Linear(obs_size, HIDDEN_SIZE),
#      nn.Sigmoid(),
#      nn.Linear(HIDDEN_SIZE, n_actions)
# )

16 4


In [132]:
objective = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

In [133]:
# Get an action
sm = nn.Softmax(dim=1)

def select_action(state):
    state_tensor = torch.FloatTensor([state])
    act_probs_tensor = sm( net(state_tensor) )
    act_probs = act_probs_tensor.data.numpy()
    act_probs = act_probs[0]
    action = np.random.choice(len(act_probs), p = act_probs )
    return action

initial_state = environment.reset()
#print(initial_state)
select_action(initial_state)
# sm = nn.Softmax(dim=1)
#    def select_action(state):
# 1:      state_t = torch.FloatTensor([state])
# 2:      act_probs_t = sm(net(state_t))
# 3:      act_probs = act_probs_t.data.numpy()[0]
# 4:      action = np.random.choice(len(act_probs), p=act_probs)
#         return action

2

In [134]:
# dummy = [1.0, 2.0, 1.0, 4.5]
# dummy_float_tensor = torch.FloatTensor(dummy)

# dummy_float_tensor.data.numpy()

In [135]:
BATCH_SIZE = 100
GAMMA = 0.9

PERCENTILE = 30
REWARD_GOAL = 0.8

from collections import namedtuple

Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [136]:
from typing import List
iter_no = 0
reward_mean = 0

full_batch  = []

batch : List[Episode] = []

episode_steps = []
episode_reward = 0.0

state = environment.reset()
counter = 0

while reward_mean < REWARD_GOAL:
    counter += 1
    # if counter >= 10:
    #     break

    action = select_action(state)
    next_state, reward, episode_id_done,_ =  environment.step(action)

    episode_steps.append( EpisodeStep(observation=state, action=action) )
    episode_reward += reward

    if episode_id_done :
        batch.append( Episode(reward=episode_reward, steps=episode_steps) )
        next_state = environment.reset()

        episode_steps = []
        episode_reward = 0.0

        if len(batch) == BATCH_SIZE:
            reward_mean = float(np.mean(list(map( lambda s : s.reward , batch))))
            elite_candidates = batch

            returnG  = list(map( lambda s : s.reward * (GAMMA ** len(s.steps) ) , elite_candidates))
            reward_bound = np.percentile(returnG, PERCENTILE)

            #z_lambda = lambda z : z.action
            #print("Elite candidates: ", list(map( lambda s: list(map( lambda z: z.action , s.steps)) ,elite_candidates)) )
            #print("Return G: ", returnG)
            print("Reward bound", reward_bound)

            train_obs = []
            train_act = []
            elite_batch = []
            for example, discounted_reward in zip(elite_candidates, returnG):
                    if discounted_reward > reward_bound:
                            train_obs.extend(map(lambda step: step.observation, example.steps))
                            train_act.extend(map(lambda step: step.action, example.steps))
                            elite_batch.append(example)
            full_batch=elite_batch
            state=train_obs
            acts=train_act

            if len(full_batch) != 0 : # just in case empty during an iteration
                 state_tensor = torch.FloatTensor(state)
                 acts_tensor = torch.LongTensor(acts)
                 optimizer.zero_grad()
                 action_scores_tensor = net(state_tensor)
                 loss_tensor = objective(action_scores_tensor, acts_tensor)
                 loss_tensor.backward()
                 optimizer.step()
                 print("%d: loss=%.3f, reward_mean=%.3f" % (iter_no, loss_tensor.item(), reward_mean))
                 iter_no += 1
            batch = []
    state = next_state
    

Reward bound 0.0
0: loss=1.427, reward_mean=0.010
Reward bound 0.0
1: loss=1.380, reward_mean=0.020
Reward bound 0.0
2: loss=1.395, reward_mean=0.020
Reward bound 0.0
3: loss=1.365, reward_mean=0.010
Reward bound 0.0
Reward bound 0.0
4: loss=1.402, reward_mean=0.010
Reward bound 0.0
5: loss=1.344, reward_mean=0.040
Reward bound 0.0
6: loss=1.387, reward_mean=0.030
Reward bound 0.0
7: loss=1.373, reward_mean=0.010
Reward bound 0.0
8: loss=1.396, reward_mean=0.010
Reward bound 0.0
9: loss=1.348, reward_mean=0.010
Reward bound 0.0
10: loss=1.350, reward_mean=0.010
Reward bound 0.0
Reward bound 0.0
11: loss=1.352, reward_mean=0.020
Reward bound 0.0
12: loss=1.383, reward_mean=0.030
Reward bound 0.0
13: loss=1.342, reward_mean=0.030
Reward bound 0.0
14: loss=1.395, reward_mean=0.010
Reward bound 0.0
15: loss=1.338, reward_mean=0.020
Reward bound 0.0
16: loss=1.387, reward_mean=0.010
Reward bound 0.0
17: loss=1.358, reward_mean=0.020
Reward bound 0.0
18: loss=1.347, reward_mean=0.030
Reward 

In [145]:
for i in range(10):
        test_env = OneHotWrapper(gym.make("FrozenLake-v1", 
           is_slippery=False))
        state= test_env.reset()
        test_env.render()
        is_done = False
        while not is_done:
                action = select_action(state)
                new_state, reward, is_done, _ = test_env.step(action)
                test_env.render()
                state = new_state
        print("reward = ", reward)


SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
reward =  1.0

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
reward =  0.0

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
